In [23]:
from pandas import Series, DataFrame
from snownlp import SnowNLP
import numpy as np
import jieba
import jieba.analyse
import pprint

In [24]:
import pandas as pd
df = pd.read_excel('C:/Users/Robin/Google Drive/Code/Data/tmall_logi_reviews.xlsx', encoding='utf-8')
df2 = pd.read_excel('C:/Users/Robin/Google Drive/Code/Data/tmall_logi_products.xlsx')

In [25]:
df.head(3)

,爬取时间(__time),爬取链接(__url),product_id,Publish Time,content,评价者(author_name),评价者会员等级(author_level),product_sku,确认收货时间(trade_end_time),卖家回复内容(seller_reply_content),append_time,append_interval,append_content,追加评价卖家回复内容(append_seller_reply_content)
0,2017-06-27 09:13:37,https://rate.tmall.com/list_detail_rate.htm?it...,521301949400,2015-08-19 10:12:40,是正品，罗技的东西还是不错的，可惜这么贵的只有一年保修。功能强大，可以用蓝牙连接，可以操作3...,t***7,天猫T3达人,套餐类型:套餐一;颜色分类:MX Anywhere2,2015-08-19 10:07:18,无,NaT,无,NaN,无
1,2017-06-27 09:13:37,https://rate.tmall.com/list_detail_rate.htm?it...,521301949400,2015-08-21 06:19:15,鼠标用起来很好，是正品，可以连接多个键盘鼠标，节省了一个端口，手感很好，比第一代好了很多，拿...,海***5,天猫T2达人,套餐类型:套餐一;颜色分类:MX Anywhere2,2015-08-21 06:14:09,无,NaT,无,NaN,无
2,2017-06-27 09:13:37,https://rate.tmall.com/list_detail_rate.htm?it...,521301949400,2015-08-22 06:47:05,很满意地一次购物，物流快，质量好，下次还会在来的,乡***街,天猫T4达人,套餐类型:官方标配;颜色分类:MX Anywhere2,2015-08-22 06:46:23,无,NaT,无,NaN,无


In [42]:
reviews = df[df['product_id'] == 25423416810]['append_content'].dropna(axis=0)
# reviews = df[df['product_id'] == 38373582263]['content'].dropna(axis=0)
# reviews = df['append_content'].dropna(axis=0)
reviews.shape

(67,)

In [43]:
import jieba.analyse
jieba.analyse.set_stop_words('C:/Users/Robin/Google Drive/Code/Project/TextMining/stoplist.txt')
jieba_text = reviews.tolist()
tags = jieba.analyse.extract_tags(str(jieba_text), topK=20, withWeight=True, allowPOS=('a,n'))
word_rank = pd.DataFrame(tags,columns=['word','rank'])
word_rank

,word,rank
0,鼠标,0.608994
1,不错,0.377686
2,滑鼠,0.261046
3,卖家,0.256945
4,滚轮,0.242187
5,左键,0.168377
6,客服,0.162809
7,垃圾,0.150045
8,电池,0.141199
9,质量,0.136034


In [44]:
from snownlp import SnowNLP
snow_reviews = ''
for review_temp in reviews:
    snow_reviews = snow_reviews + '。' + str(review_temp)
s = SnowNLP(snow_reviews)
s.summary(15)

['卖家也不给退货',
 '不知道是因为太滑了还是不灵敏',
 '贪便宜在这家买没好货',
 '就是不好用',
 '不要买这家也更不买这款',
 '建议都不要买',
 '质量很好',
 '但个人感觉不像真货：滚轮滚起来太轻',
 '卖家态度也很差',
 '很好用',
 '很好用',
 '很好',
 '很好',
 '不知道是太灵敏了还是跳帧',
 '鼠标好用']

In [7]:
from pandas import Series, DataFrame
review_sentiments = []
for review in reviews:
    try:
        s = SnowNLP(str(review))
        review_sentiment =s.sentiments
        review_sentiments.append(review_sentiment)
    except:
        review_sentiments.append('NaN')
review_obj = DataFrame({'review':reviews,
                        'sentiment':review_sentiments})

In [10]:
print (review_obj.sort_values('sentiment',axis=0)[0:10])

                                                   review     sentiment
110785  <b></b>差<b></b>评<b></b>！<b></b>差<b></b>评<b></b...  0.000000e+00
117427  <b></b>便<b></b>宜<b></b>没<b></b>好<b></b>货<b></b...  0.000000e+00
106144  键盘买回来第一次用挺正常，第二次用就坏了！灯都不亮了！驱动神马的也装了，就是检测不到键盘！客...  0.000000e+00
110494  我收到东西 掌托 是坏的 跟客服说 客服根本就不理我 一直弄了 两天 我收货当天就给客服发了...  0.000000e+00
123683  <b></b>收<b></b>到<b></b>键<b></b>盘<b></b>后<b></b...  0.000000e+00
106790  买的东西也不便宜了 怎么那么吃 对待买家呢 买的时候痛快 收到之后售后的支持 真是太差了 我...  2.220446e-16
120065  <b></b>买<b></b>了<b></b>手<b></b>感<b></b>不<b></b...  1.243450e-14
106786  客服态度非常非常的差，买重了东西，分两个包裹寄给我的，我把一个包裹里所有的东西都寄回去了，跟...  8.748557e-14
116389  <b></b>首<b></b>先<b></b>，<b></b>我<b></b>承<b></b...  2.582379e-13
110059  买回来打开掌托就是坏的，联系客服就说没收到快递时候验货，还外观不保，我就想问一个键盘除了外观...  3.188561e-13


In [9]:
rev_len = len(reviews)
n1 = 0
for review_temp in reviews:
    token1  = '麻烦'
    try:
        if token1 in review_temp:
            print (review_temp)
            n1 += 1
    except:
        pass

print (token1,n1,rev_len,n1/rev_len)


快递超给力啊，13号下的订单，14号就送到了，键盘什么都很喜欢，服务也超赞，就是设置对于我这样的小白有点儿麻烦，总之给店家点个赞
东西收到了，物流超级快，头天下单今天就能到！键盘也不错，很炫，毕竟钱在这！第一次买这么贵的键盘，开始还有点犹豫，后来还是咬牙买了。这里不得不说客服，那是真心不错，发货后两次因个人原因麻烦客服，都帮我处理了，大赞一个！唯一美中不足的就是我没有要鼠标，现在除了后悔还是后悔?要是还能按这个优惠价给我发个鼠标就完美了！
键盘超级棒，5星不用说，就是一键宏?设置有些麻烦了
本人是不怎么用键盘的，所以机械键盘与否给我的感觉真的不是很明显，不过舍友拿来玩飞车说比普通键盘要更好用，键盘很有分量，开了背光灯后很好看，不过只能调节模式而不能调节亮度强弱，这点有点小败笔，因为灯光太亮了会分散你的注意力，但总体好评吧。慢慢适应，就不那么麻烦退货了。
小键盘的0键卡槽断裂，快递的锅，退换货太麻烦只能将就了
键盘超好用，但是，第一次购买这种键盘的人，那几排按键，会对你造成不小的麻烦，必须要熟悉，而且，键位和原来的键盘会有不小的间距差别。比如原来我爱数字一号键。可以按到，现在我按的可能就是2号！打字的声音是非常好的，比那些机械键盘的声音，要更柔和。不屑那种机械键盘按下去霹雳拍啦的，这种有那种薄膜键盘的感觉，但比薄膜键盘的声音更好。
买了了自己下载驱动，调试也是自己弄，很麻烦，但是习惯好评
这个键盘手感比cherry轴舒服 打字顺手 游戏不会累 据说按键寿命是7000w次 左边的G热键感觉不是很实用 最起码对于我 我是玩坦克的 偶尔手会AFK 摸回去的时候总是把热键当做最左边的按键这样就很尴尬了 这款键盘我最喜欢的两点设计就是音量旋钮 还有手机支架 音量旋钮拨动起来手感很顺滑 而且灵敏 手机支架就厉害了 虽然是一个很小的结构 但是玩游戏的时候来微信什么的可以很快看到 又不耽误事儿 不过要特别提醒看官一下 桌面小用笔记本的慎重选择 为什么呢 因为手托是键盘主体加盖板两部分设计 如果盖板悬在桌子边上 很容易一抬手就把盖板碰翻掉 而罗技的工学设计盖板下部居然只用一个小塑料一体的卡扣锁住 碰开一两次卡扣就会断！！！ 为什么这么肯定？ 因为我的键盘刚拆箱的时候它就已经是断的！由于这边快递麻烦 就没有过多纠结 不过这个盖板可是真心没单独卖的 真是要吐槽黑龙江亚布力镇的顺丰 除了顺丰

In [20]:
product_id = list(set(df.product_id))

n1_list = []
n1_rate_list = []
len_list = []
pro_name_list = []
for pro_id in product_id:
    pro_reviews = df[df['product_id'] == pro_id]['content'].dropna(axis=0)
    rev_len = len(pro_reviews)
    n1 = 0
    for review_temp in pro_reviews:
        token1  = '男朋友'
        token2 = '老公'
        try:
            if token1 in review_temp:
                n1 += 1
            elif token2 in review_temp:
                n1 += 1
        except:
            pass
    pro_name = df2[df2['product_id'] == pro_id]['name']
    n1_rate = n1/rev_len
    n1_list.append(n1)
    n1_rate_list.append(n1_rate)
    len_list.append(rev_len)
    pro_name_list.append(pro_name)
pro_obj = DataFrame({'pro_id':product_id,
            'token1':n1_rate_list,
            'n1':n1_list,
            'rev_len':len_list,
            'pro_name':pro_name_list})
pprint.pprint (pro_obj.sort_values('token1',axis=0, ascending=False)[0:15])

     n1        pro_id                                           pro_name  \
331   1  551063954708  264    送礼顺丰 罗技G Pro 有线RGB炫彩背光87键专业电竞游戏机械键盘
Nam...   
377   5   26037980640  494    罗技G430游戏耳机耳麦克风笔记本台式机头戴式7.1cf守望先锋耳麦
Name...   
140   3  536254786111  112    顺丰罗技G610茶轴/青轴/红轴机械键盘带背光键盘无冲游戏竞技键盘
Name:...   
484   1  551482279831  291    顺丰罗技G413有线背光宏编程104键守望先锋/LOL电竞游戏机械键盘
Nam...   
23    4  540780202055  550    Logitech/罗技 G231游戏耳机头戴式有线电竞台式电脑通用耳机带麦
N...   
260   2   45188875257  97    包邮送桌垫 Logtiech/罗技 MK520无线键鼠套装笔记本键盘鼠标优联
N...   
135   2  527943993896  429    顺丰送礼罗技G402有线游戏鼠标 发光魔兽电竞机械守望先锋LOL CF
Nam...   
55    2  527937118202  445    顺丰罗技G910炫彩有线游戏机械键盘G402/g502鼠标LOL键鼠套装G63...   
183   1  543127751427  247    包邮罗技G213炫彩游戏有线薄膜键盘竞技有线机械鼠标套装手感编程
Name: ...   
313   2  531317138626  462    顺丰罗技G900 RGB炫光无线鼠标有线电竞游戏LOL台式电脑无延迟
Name...   
48    3  542686888117  150    顺丰 罗技G102 游戏鼠标RGB炫光 LOL守望先锋CF 宏编程 有线鼠标
...   
32    1  524640444513  263    现货顺丰 Logitech/罗技 G633 7.1环绕声PC/PS4带麦克风游...   
129   1  539

In [9]:
all_reviews = df['content'].dropna(axis=0)
